In [13]:

# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import zeros
from numpy import ones
import numpy as np
import pandas as pd
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
import os
#from torch.utils.data import Dataset, DataLoader

class Data():
    def __init__(self, split, subject,  dataset,path):
        
        assert split in ['train', 'test']
        
        self.dataset_root = path
        self.dataset_path = os.path.join(self.dataset_root, dataset)
        self.subject = subject        
        self.split = split
        self.x = None
        self.y = None
        


    def load(self):

        self.x = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'X_train.npy', allow_pickle=True)

        if self.split == 'train':
            self.y = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'y_train.npy', allow_pickle=True)
        else:
            self.y = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'y_test.npy', allow_pickle=True)
        
        
        
        
        
        return self.x, self.y
    
    
def y_translator(y,subject,dataset):
    # Define your letters
    letters = set(y)
    y_train_original = y
    # Create a dictionary to map letters to numbers
    letter_to_number = {letter: i for i, letter in enumerate(letters)}

    path = f"{dataset}/dictionary/"

    os.makedirs(path, exist_ok=True)

    np.save(f'{path}Subject_{subject}.npy', letter_to_number) 

    # Test the conversion
    y_train_new=[]
    for letter in y_train_original:
        input_letter = letter
        if input_letter in letter_to_number:
                y_train_new.append(letter_to_number[input_letter])
            #print(f"The number corresponding to {input_letter} is {letter_to_number[input_letter]}")
        else:
            print(f"{input_letter} is not a valid letter.")
    return(np.array(y_train_new))


def load_real_samples(split,subject,dataset):

    test = Data(split,subject=subject,dataset=dataset,path=f'')

    X,y = test.load()
    y = y_translator(y,subject,dataset)
    y = y.reshape(-1, 1)
    return [X,y]

In [14]:
dataset = load_real_samples("train",subject='02',dataset="FALL-mag")

In [23]:
X = dataset[0]
y = dataset[1] # y apparently only have 3 classes.

In [17]:
X.shape

(27017, 50, 3)

In [18]:
y.shape

(27017, 1)

In [26]:
# So, I have a X with shape (27017, 50, 3) and y with shape (27017, 1).
# I need a way to augment that through hugging face API.
# Then I need another time series classificator and test the performance on the augmented data and the not augmented data. Then collect the results.
# That is pretty much it, goodnight me. See ya tomorrow again.



First, I need to transform the data into a NLP format through Hugging Face lib's. Then, I'll need to adapt Hugging face augmentation to it.

In [28]:
!pip install torch transformers datasets tsaug scikit-learn nlpaug


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.8 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.8 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 43.8/43.8 kB 308.0 kB/s eta 0:00:00
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.1 MB 1.9 MB/s eta 0:00:05
    --------------------------------------- 0.2/9.1 MB 1.5 MB/s eta 0:00:06
   - -------------------------------------- 0.3/9.1 MB 1.9 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.1 MB 2.9 MB/s eta 0:00:04
   --- -------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\PC\\AppData\\Roaming\\Python\\Python310\\site-packages\\safetensors\\tensorflow.py'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
  Using cached tsaug-0.2.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached nlpaug-1.1.11-py3-none-any.whl.metadata (14 kB)
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached regex-2024.5.15-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.3-cp310-none-win_amd64.whl.metadata (3.9 kB)
  Using cached pyarrow-16.1.0-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fss

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.42.1 requires cloudpickle, which is not installed.
tsgm 0.0.4 requires networkx, which is not installed.
tsgm 0.0.4 requires seaborn, which is not installed.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.2.2 which is incompatible.


In [29]:
import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = self.X[idx]
        label = self.y[idx]
        
        if self.transform:
            sample = self.transform(sample)
        
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


ImportError: cannot import name 'Dataset' from 'torch.utils.data' (unknown location)